Подсчёт количества правильных семейств из 3 функций с помощью алгоритма построения правильного семейства N+1 функций из правильных семейств N функций

In [1]:
import itertools as it
from tqdm import tqdm
import numpy as np
import multiprocessing
from ipynb.fs.full.functions import func
from math import ceil

Генерация наборов значений переменных x и y вместе с порядковым номером набора

In [2]:
num_args_2 = tuple(enumerate(it.product((0,1), repeat = 2)))
num_args_2

((0, (0, 0)), (1, (0, 1)), (2, (1, 0)), (3, (1, 1)))

Генерация различных пар наборов

In [3]:
pairs_2 = tuple(it.combinations(num_args_2, 2))
pairs_2

(((0, (0, 0)), (1, (0, 1))),
 ((0, (0, 0)), (2, (1, 0))),
 ((0, (0, 0)), (3, (1, 1))),
 ((1, (0, 1)), (2, (1, 0))),
 ((1, (0, 1)), (3, (1, 1))),
 ((2, (1, 0)), (3, (1, 1))))

Выводит индексы позиций, в которых отличаются 2 вектора

In [4]:
def diff_idx(x,y):
    return (i for (i,(a,b)) in enumerate(zip(x,y)) if a!=b)

Проверка на правильность по определению (на вход подаётся набор из двух функций, каждая из которых представлена вектором значений на каждом из наборов переменных)

In [5]:
def func_2(f):
    for p in pairs_2:
        flag = False
        for idx in diff_idx(p[0][1],p[1][1]):
            if f[idx][p[0][0]] == f[idx][p[1][0]]:
                flag = True
                break
        if not flag:
            break
    return flag

Так как i-я функция не зависит от i-ой переменной, то нам достаточно знать значения функции на 2/4 наборах, остальные значения определяются по этим 2 наборам. 

In [6]:
def f_2(elem, idx):
    l = [0] * 4
    if idx == 0:
        l[0] = l[2] = elem[0]
        l[1] = l[3] = elem[1]
    else:
        l[0] = l[1] = elem[0]
        l[2] = l[3] = elem[1]
    return tuple(l)

f_0 - итератор, состоящий из функций, не зависящие от x, а f_1 - итератор, состоящий из функций, не зависящие от y. iterator_2 - итератор, состоящий из элементов декартового произведения f_0 и f_1, то есть семейства-кандидаты на правильность. Итератор существует до тех пор, пока не будут перечислены все его элементы, вызываемые методом next. Если элементы в итераторе закончились, то он бросает исключение. Использовал итераторы для экономии памяти.

In [7]:
iter_2 = it.product((0,1), repeat = 2)
f_0 = map(lambda p: f_2(p, 0), iter_2)
iter_2 = it.product((0,1), repeat = 2)
f_1 = map(lambda p: f_2(p, 1), iter_2)
iterator_2 = it.product(f_0, f_1)

Проходим в цикле по всем семействам и подсчитываем правильные (переменная s) и сохраняем правильные семейства в переменную collect.

In [8]:
s=0
collect = []
for _ in tqdm(range(16)):
    el = next(iterator_2)
    if func_2(el):
        s+=1
        collect.append(el)
print(s)

100%|██████████| 16/16 [00:00<00:00, 49673.47it/s]

12


In [9]:
collect

[((0, 0, 0, 0), (0, 0, 0, 0)),
 ((0, 0, 0, 0), (0, 0, 1, 1)),
 ((0, 0, 0, 0), (1, 1, 0, 0)),
 ((0, 0, 0, 0), (1, 1, 1, 1)),
 ((0, 1, 0, 1), (0, 0, 0, 0)),
 ((0, 1, 0, 1), (1, 1, 1, 1)),
 ((1, 0, 1, 0), (0, 0, 0, 0)),
 ((1, 0, 1, 0), (1, 1, 1, 1)),
 ((1, 1, 1, 1), (0, 0, 0, 0)),
 ((1, 1, 1, 1), (0, 0, 1, 1)),
 ((1, 1, 1, 1), (1, 1, 0, 0)),
 ((1, 1, 1, 1), (1, 1, 1, 1))]

Конвертируем collect в numpy-массив для удобства.

In [10]:
collect = np.array(collect)

In [11]:
collect

array([[[0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 1, 1]],

       [[0, 0, 0, 0],
        [1, 1, 0, 0]],

       [[0, 0, 0, 0],
        [1, 1, 1, 1]],

       [[0, 1, 0, 1],
        [0, 0, 0, 0]],

       [[0, 1, 0, 1],
        [1, 1, 1, 1]],

       [[1, 0, 1, 0],
        [0, 0, 0, 0]],

       [[1, 0, 1, 0],
        [1, 1, 1, 1]],

       [[1, 1, 1, 1],
        [0, 0, 0, 0]],

       [[1, 1, 1, 1],
        [0, 0, 1, 1]],

       [[1, 1, 1, 1],
        [1, 1, 0, 0]],

       [[1, 1, 1, 1],
        [1, 1, 1, 1]]])

Перевод числа от 0 до 4 в двоичную систему счисления (выходные числа длины 2 для удобного представления в виде наборов переменных)

In [12]:
def convert_to_base_2(x):
    if x == 0:
        return np.base_repr(x, base=2, padding=2)
    elif x == 1:
        return np.base_repr(x, base=2, padding=1)
    else:
        return np.base_repr(x, base=2)      

Перевод строкового представления числа в двоичной системе счисления в набор значений переменных на примере числа 3

In [17]:
np.array(list(convert_to_base_2(3)), dtype=int)

array([1, 1])

Набор, каждый элемент которого состоит из 2 правильных семейств порядка 2

In [18]:
set_of_fam = np.array(list(it.product(collect, repeat=2)))

Проверка условия из леммы 3 (возвращает True/False в зависимости от того, нужно ли соединять вершины графа(наборы переменных) ребром).

In [20]:
def check_edge(s, k, g):
    flag = False
    for (i, j) in it.permutations(range(2), 2):
        if set(diff_idx(np.array(list(convert_to_base_2(s)), dtype=int),\
                        np.array(list(convert_to_base_2(k)), dtype=int))).\
           issubset(set(diff_idx(g[i][:, s],g[j][:, k]))):
            flag = True
            break
    return flag

Поиск всех вершин, находящихся в одной компоненте связности с i-ой вершиной

In [21]:
def find_comp(i, comp, use_vert, graph):
    for j in range(4):
        if graph[i, j] == 1 or graph[j, i] == 1:
            if j not in use_vert:
                use_vert.append(j)
                comp.append(j)
                find_comp(j, comp, use_vert ,graph)

Поиск всех компонент связности в графе. Выдает количество компонент связности и сами компоненты

In [22]:
def find_components(graph):    
    use_vert = []
    components = []
    for i in range(4):
        if i not in use_vert:
            use_vert.append(i)
            comp = [i]
            find_comp(i, comp, use_vert ,graph)
            components.append(comp) 
    return len(components), components

Построение графа (заполняется только верхняя диагональ)

In [24]:
def build_graph(fam):
    graph = np.zeros((4,4), dtype=int)
    for (i, j) in it.combinations(range(4), 2):
        if check_edge(i, j, fam):
            graph[i][j] = 1
    return graph

Подсчет количества всевозможных правильных семейств порядка 3, построенных по набору из 2 правильных семейств порядка 2. Для каждого такого набора считаем количество всевозможных доопределений третьей функции на каждой компоненте связности графа.

In [25]:
count = 0
for fam in tqdm(set_of_fam):
    graph = build_graph(fam)
    count += 2 ** find_components(graph)[0]
print(count)

100%|██████████| 144/144 [00:00<00:00, 4626.64it/s]

744


Реализация унарных функций 2-значной логики I_0,...,I_{k-1}

In [26]:
def I_func(x, j):
    return 1 if x==j else 0

Основный алгоритм построения различных семейств порядка 3, который получает на вход 2 правильных семейства порядка 2 и в результате выполнения функции наполняет список lst правильными семействами порядка 3, полученными путём всевозможных доопределений функции g на всех компонентах связности графа

In [30]:
set_of_fam[0].shape

(2, 2, 4)

In [31]:
def build_all_fam(fam, comp_num, components, lst):
    short_fam = tuple(tuple(max(min(I_func(x,0), fam[0,j,y]),\
                                min(I_func(x,1), fam[1,j,y])) for y in range(4) for x in range(2)) for j in range(2))
    var_of_redef = it.product((0,1), repeat=comp_num)
    for var in var_of_redef:
        g = [0] * 8
        for (i,comp) in enumerate(components):
            for arg in comp:
                g[2*arg:2*arg+2] = [var[i]]*2
        new_fam = short_fam + (tuple(g),)
        lst.append(new_fam)

Строим правильные семейства с помощью алгоритма, использующего граф (реализован в функции build_all_fam), который получает на вход 2 правильных семейства порядка 2 и выдает все построенные семейства порядка 3, полученные с помощью различного доопределения функции на компонентах связности графа

In [32]:
lst = []
for fam in tqdm(set_of_fam):
    graph = build_graph(fam)
    comp_num, components = find_components(graph)
    build_all_fam(fam, comp_num, components, lst)

100%|██████████| 144/144 [00:00<00:00, 3715.68it/s]


In [33]:
lst = tuple(lst)

In [34]:
len(lst)

744

С помощью параллельных вычислений проверяем, что все полученные семейства являются правильными по определению

In [35]:
s = 0
for fam in tqdm(lst):
    if func(fam):
        s+=1
print(s)        

100%|██████████| 744/744 [00:00<00:00, 19799.77it/s]

744


Конвертируем кортеж с правильными семействами в numpy-массив

In [58]:
lst = np.array(lst)

Записываем результат в файл result.npz с применением сжатия

In [59]:
np.savez_compressed('all_fams_2', lst)